In [3]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [4]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-14 21:52:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.89MB/s    in 0.2s    

2023-03-14 21:52:37 (4.89 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [5]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles


url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [7]:
# Get the number of rows in the DataFrame.

user_data_df.count()

2557288

# Transform the Data

## Create the "review_id_table".

In [8]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.


review_id_table = user_data_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date')])
review_id_table = review_id_table.withColumnRenamed("to_date(review_date)","review_date")
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RED72VWWCOS7S|   32787517|B008HDQYLQ|     348668413| 2015-08-31|
| RZHWQ208LTEPV|   16374060|B005OBZBD6|     264704759| 2015-08-31|
|R37LBC3XAVLYOO|    9984817|B00RQL8U2G|      95173602| 2015-08-31|
|R3L7XJMA0MVJWC|   12635190|B0081SBO4Y|     835659279| 2015-08-31|
|R2I2GHSI7T1UBN|   43905102|B008E6OK3U|     539243347| 2015-08-31|
|R2GFFKHK4I6VMX|   52596997|B00W6NTULY|     337446474| 2015-08-31|
|R1R0UDX2XAN1S4|   43871104|B00GXUMYKA|     468857193| 2015-08-31|
|R22C8FMBSTFRY8|   11346008|B005EIX8JS|     125753094| 2015-08-31|
|R118NNIQ75XPGO|   49206471|B000HJBKMQ|     834273114| 2015-08-31|
|R30HYXHZQ49621|   37596267|B004LY59V6|     612086079| 2015-08-31|
|R3EMLKY0GF1E90|   31554283|B00CAVM85M|     280334010| 2015-08-31|
|R23BX7EGJMGQJR|   43211735|B00DP6X1LG|     233116679| 2015-08

## Create the "products" Table

In [9]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = user_data_df.select(['product_id', 'product_title'])
products_df = products_df.dropDuplicates(['product_id'])
products_df = products_df.dropDuplicates(['product_title'])

products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00N15UIXA|!!Final Clearance...|
|B007QW5OA6|!!NEW 2012 CHOICE...|
|B00BNZFK42|" All We Do Is Be...|
|B00HSK3KXE|" Amazing Grace "...|
|B00HABAZYS|" American Eagle ...|
|B00G350J84|" BLAZE ORANGE CA...|
|B00HTEJAJC|" BUNNY & STRIPES...|
|B00B65K4YU|" Blue Crab Welco...|
|B004I2MP0O|" Bluebirds Geran...|
|B00H9EOJM0|" Bunny & Basket ...|
|B00H9EOKHY|" Bunny Stripes "...|
|B00H9EOK92|" Bunny Stripes -...|
|B00SOXTY1G|" Burlap Pot of D...|
|B00ND9WAL2|" CABIN FEEDER " ...|
|B00HBFZI86|" CARDINALS & hYD...|
|B00NR3XCJI|" CHEVRON PINEAPP...|
|B00NR3XVIK|" CHEVRON PINEAPP...|
|B00NR44SPY|" CHEVRON PINEAPP...|
|B00LV505YE|" CHRISTMAS SEASO...|
|B00LW23V42|" COLORFUL PUMPKI...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [10]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

customers_df = user_data_df.groupby("customer_id").count()
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46263557|             1|
|   44418648|             1|
|   34220092|             1|
|   10770444|             1|
|   12819130|             1|
|   24692214|             3|
|   51981844|             3|
|   43622169|             1|
|   43097449|             1|
|   45978717|             1|
|   20227781|             1|
|   11487525|             4|
|   31169607|             1|
|   22552080|             2|
|   15829398|             2|
|   46909180|             1|
|     198573|             3|
|   33383395|             1|
|    2934525|             3|
|    2333626|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [11]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 

vine_df = user_data_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'] )

vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RED72VWWCOS7S|          1|            2|          8|   N|
| RZHWQ208LTEPV|          5|            0|          0|   N|
|R37LBC3XAVLYOO|          5|            4|          5|   N|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|
|R22C8FMBSTFRY8|          5|            2|          2|   N|
|R118NNIQ75XPGO|          3|            0|          0|   N|
|R30HYXHZQ49621|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



# Load

In [17]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb1.cigtzod05pek.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "381634Cm!", "driver":"org.postgresql.Driver"}

In [19]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [21]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)


In [22]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)